# 为什么需要Type hint？
让我们从最简单的例子开始：

In [1]:
def get_full_name(first_name, last_name):
    full_name = first_name.title() + " " + last_name.title()
    return full_name

print(get_full_name("john", "doe"))

John Doe


现在，想象你在重新写这几行代码。你想要找到能让字母大写的函数。***但你忘记它是什么了。***

In [2]:
def get_full_name(first_name, last_name):
    full_name = first_name

尝试下在`first_name`后加一个点，看看IDE会给你提示些什么。你会发现只有一个叫`title`的东西。这是上一个code block遗留下来的用法，而且用法还是错的。

**那怎么让IDE知道你这个变量是一个字符串，并且提示你字符串该有的方法呢？**

In [3]:
def get_full_name(first_name: str, last_name: str):
    full_name = first_name

你会发现在argument后加上了`: str`后，IDE就能正确的帮你提示了。

`[var]: type`的表达方式就是type hint。

再来个例子。还是之前的`get_full_name()`。如果你没有定义type hint，用户输入了一个不是字符串的argument，会发生什么呢？

In [5]:
def get_full_name(first_name, last_name):
    full_name = first_name.title() + " " + last_name.title()
    return full_name


print(get_full_name(111, "doe"))

AttributeError: 'int' object has no attribute 'title'

报错。如果加了type hint呢？

In [6]:
def get_full_name(first_name: str, last_name: str):
    full_name = first_name.title() + " " + last_name.title()
    return full_name


print(get_full_name(111, "doe"))

AttributeError: 'int' object has no attribute 'title'

他不会像你所期待的那样。它还是在报相同的错。**根本原因在于：Python的Type hint真的只是“hint"，给编辑器作个提示罢了。至于Type "check"，Python说不关我事。**

这时候有请我们的下一位主角登场：

In [7]:
!pip install pydantic


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


Pydantic是个神奇的库。官网简介，Pydantic is the most widely used data validation library for Python。它通过一系列自定义的class，实现了Type Validation。

让我们再尝试一遍：

In [8]:
from pydantic import BaseModel

class NameType(BaseModel):
    first_name: str
    last_name: str
    
def get_full_name(name: NameType):
    full_name = name.first_name.title() + " " + name.last_name.title()
    return full_name


print(get_full_name(NameType(first_name=1111, last_name="Doe")))

ValidationError: 1 validation error for NameType
first_name
  Input should be a valid string [type=string_type, input_value=1111, input_type=int]
    For further information visit https://errors.pydantic.dev/2.4/v/string_type

同样是报错，但这一报错指明了这是用户的错，不是软件的锅。更重要的是，Type Validation可以尽早地发现Type Error，直接在函数运行前就阻止了可能带来的隐患。

俗话说“Never trust user input"，Type Validation是一个非常好的Practice。很多软件正是因为这方面的疏忽，导致一些别有用心的用户输入一些精心设计的内容，导致出现问题。


---

接下来让我们再看几个复杂的type。（Pydantic先放到一边，无非是多一步定义Class。**[如果以下的代码报错，请更新你的Python版本]**

**1. List**

In [ ]:
def process_items(items: list[str]):
    for item in items:
        print(item)

它的意思是：这个叫`items`的变量是个`list`，这个`list`里面都是`str`。

**2. Tuple & Set**

In [ ]:
def process_items(items_t: tuple[int, int, str], items_s: set[bytes]):
    return items_t, items_s

与`list`用法类似。
1. 变量`items_t`是一个`tuple`，内含三个元素，分别是`int`, `int`和`str`。(这一定义方式也适用于`list`和`set`，反之亦然）
2. 变量`items_s`是一个`set`，里面的元素全是`bytes`。

**3. Dict**

In [ ]:
def process_items(prices: dict[str, float]):
    for item_name, item_price in prices.items():
        print(item_name)
        print(item_price)

`dict`也很简单: `dict[key_type, value_type]`
在这一例子中，`prices`是一个`dict`。这个字典中，键一定是`str`，值一定是`float`。

**4. Union**

In [ ]:
def process_item(item: int | str):
    print(item)

`Union`不是一个type，它是一个用法。它的意思是：`item`既可以是`int`，也可以是`str`。

union还有一个特殊用法：

In [ ]:
def process_item(item: int | None):
    print(item)

表示process_item既可以是`int`，也可以是空值。这里有一个容易搞混的地方在于：它可以是`None`不代表它可以不传入！

In [9]:
process_item()

NameError: name 'process_item' is not defined

以上代码是会报错的！

以下是正确代码：

In [ ]:
process_item(None)

这里再添加一点。函数参数的默认值。

In [ ]:
def test(arg1 = "Hi!"):
    print(arg1)
    
test()

能不传入参数的是这种情况。在定义函数时已经给出默认值，即使调用时不传参也可以继续执行。

把它和Type Hint结合一下：

In [ ]:
def test(arg1: str = "Hi!"):
    print(arg1)
    
test()

In [ ]:
def test(arg1: str | int = "Hi!"):
    print(arg1)
    
test()

## **练习：**

In [ ]:
def get_max_score(score_dic):
    """
    返回学生考试成绩的最高分的科目和分数
    :param score_dic:
    :return:
    """
    max_score = 0
    max_score_course = ''
    for course, score in score_dic.items():
        if score > max_score:
            max_score = score
            max_score_course = course

    return max_score_course, max_score

dic = {
    'chinese': 90,
    'math': 97,
    'english': 98
}

course, score = get_max_score(dic)
print(course, score)

请你根据以上的`dic`，用Pydantic定义一个成绩的Model，改写以上代码，给它加上严格的Type Check。成绩是Optional，如果未输入则默认为0。
如果能自查文档，自己写一个validator，验证成绩在0-100分之间，那就更好啦！:-）